In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import torch
import torchvision
from torch.utils.data import TensorDataset,DataLoader
from torchvision import transforms

['seg_test', 'seg_train', 'seg_pred']


In [2]:
comp = transforms.Compose([
        transforms.Resize(size=(150,150)),
        transforms.ToTensor()])

train_ds = torchvision.datasets.ImageFolder("../input/seg_train/seg_train/", transform=comp)
train_loader = DataLoader(train_ds,batch_size=128, shuffle=True)


In [3]:
val_dataset = torchvision.datasets.ImageFolder("../input/seg_test/seg_test/", transform=comp)
val_ds, test_ds = torch.utils.data.random_split(val_dataset, [1500, 1500])
val_loader = DataLoader(val_ds, batch_size=128)
test_loader = DataLoader(test_ds, batch_size=128)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import math
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.properties = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=75, kernel_size=3, padding=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=75, out_channels=100, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(10),
            nn.ReLU(),
            nn.Conv2d(in_channels=100, out_channels=120, kernel_size=3),
            nn.MaxPool2d(3)
        )
            
        self.estimator = nn.Sequential(
            nn.Linear(1920,500),
            nn.ReLU( inplace=True ),
            nn.Linear(500,6)
        )
        
        
    def train(self,train_loader,val_loader,epoch,waiting,optimizer):
        self.cuda()
        best_val_loss=1000
        crit = nn.CrossEntropyLoss()
        for i in range(epoch):
            train_loss = 0
            val_loss = 0
            for xx,yy in train_loader:
                xx = xx.cuda()
                yy=yy.cuda()
                optimizer.zero_grad()
                y_pred = self.forward(xx)
                loss = crit(y_pred,yy)
                train_loss += loss
                loss.backward()
                optimizer.step()
            train_loss = train_loss/len(train_loader)
            with torch.no_grad():
                for xx,yy in val_loader:
                    xx, yy = xx.cuda(), yy.cuda()
                    y_pred = self.forward(xx)
                    loss = crit(y_pred,yy)
                    val_loss += loss
                val_loss = val_loss/len(val_loader)
                
                if best_val_loss>val_loss:
                    torch.save(self.state_dict(), "../best_model.py")
                    best_val_loss = val_loss
                    wait=waiting
                else:
                    wait -=1
                    if wait==0:
                        break
            print("train loss:", float(train_loss), "___best val loss:",float(best_val_loss), "___remaining:", wait)
    
    def forward(self, x):
        return  self.estimator(self.properties(x).view(x.size(0), -1))

In [5]:
clf = Model()
optimizer = torch.optim.Adam(clf.parameters(), lr=0.001)
clf.train(train_loader,val_loader,10,4,optimizer)

train loss: 1.0042239427566528 ___best val loss: 0.7329196929931641 ___remaining: 4
train loss: 0.6528664231300354 ___best val loss: 0.5280207395553589 ___remaining: 4
train loss: 0.5159654021263123 ___best val loss: 0.4736877679824829 ___remaining: 4
train loss: 0.4336874783039093 ___best val loss: 0.42642277479171753 ___remaining: 4
train loss: 0.3823886513710022 ___best val loss: 0.42642277479171753 ___remaining: 3
train loss: 0.35094696283340454 ___best val loss: 0.42642277479171753 ___remaining: 2
train loss: 0.2883365750312805 ___best val loss: 0.4241550862789154 ___remaining: 4
train loss: 0.25578823685646057 ___best val loss: 0.4241550862789154 ___remaining: 3
train loss: 0.21762990951538086 ___best val loss: 0.413249135017395 ___remaining: 4
train loss: 0.16955089569091797 ___best val loss: 0.413249135017395 ___remaining: 3


In [6]:
from sklearn.metrics import classification_report
clf.load_state_dict(torch.load("../best_model.py"))
y_true = []
y_pred = []
for xx,yy in test_loader:
    out = clf.forward(xx.cuda())
    for i in out:
        y_pred.append(int(i.argmax()))
    for i in yy:
        y_true.append(int(i))
print(classification_report(y_pred,y_true))

              precision    recall  f1-score   support

           0       0.84      0.81      0.82       247
           1       0.89      0.98      0.93       215
           2       0.72      0.87      0.79       227
           3       0.85      0.76      0.80       261
           4       0.91      0.82      0.86       288
           5       0.84      0.84      0.84       262

   micro avg       0.84      0.84      0.84      1500
   macro avg       0.84      0.85      0.84      1500
weighted avg       0.85      0.84      0.84      1500

